## Extracción de datos estática

### Acceso al sitio

In [0]:
%pip install beautifulsoup4

Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
#Importar librerías.
import numpy as np
import pandas as pd

from bs4 import BeautifulSoup
import requests
import datetime
import pickle

In [0]:
#URL de Fuente: TLC Trip Record Data.
tlc_url = "https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page"

#Solicitar URL y obtener objeto de respuesta.
tlc_respuesta = requests.get(tlc_url)

#Analizar el texto obtenido.
tlc_sopa = BeautifulSoup(tlc_respuesta.text, 'html.parser')

#Encontrar todos los hipervínculos presentes en la página web.
tlc_vinculos = tlc_sopa.find_all('a')

### Descarga

In [0]:
#Descarga inicial.
#Yellow Taxi Trip Records.

años = ['2022']

i = 0
d_amarillo = {}
dh_amarillo = {}

for vinculo in tlc_vinculos:
    if(
        ('yellow' in vinculo.get('href', [])) and
        ('.parquet' in vinculo.get('href', []))
    ):
        for año in años:
            if (año in vinculo.get('href', [])):
                i += 1
                hipervinculo = vinculo.get('href', [])
                mes = hipervinculo[69]+hipervinculo[70]
                print('Elemento', i, 'leído.')
                #Leer el vínculo y asignarlo a un diccionario.
                d_amarillo["yellow_{0}_{1}".format(año, mes)] = "yellow_{0}_{1} = pd.read_parquet('{2}')".format(año, mes, hipervinculo)
                print('Elemento', i, 'escrito en diccionario.')

print('Todos los elementos se escribieron en el diccionario amarillo.')

Elemento 1 leído.
Elemento 1 escrito en diccionario.
Elemento 2 leído.
Elemento 2 escrito en diccionario.
Elemento 3 leído.
Elemento 3 escrito en diccionario.
Elemento 4 leído.
Elemento 4 escrito en diccionario.
Elemento 5 leído.
Elemento 5 escrito en diccionario.
Elemento 6 leído.
Elemento 6 escrito en diccionario.
Elemento 7 leído.
Elemento 7 escrito en diccionario.
Elemento 8 leído.
Elemento 8 escrito en diccionario.
Elemento 9 leído.
Elemento 9 escrito en diccionario.
Elemento 10 leído.
Elemento 10 escrito en diccionario.
Elemento 11 leído.
Elemento 11 escrito en diccionario.
Elemento 12 leído.
Elemento 12 escrito en diccionario.
Todos los elementos se escribieron en el diccionario amarillo.


Acceso a Blob Storage

In [0]:
#Datos de acceso
storage_account_name = "pfcrudo"
storage_account_key = "ASY9nV2+pddQN2Y8AUZef5m4JtdidWOa5RW+fNIkj2HE60F5s95ueJraFC2/Jv1KulbyL5hwcYk6+AStCn7cjw=="

In [0]:
#Variable de configuración Spark
spark.conf.set(f"fs.azure.account.key.{storage_account_name}.blob.core.windows.net", storage_account_key)

Descarga de todo lo contenido en el diccionario

In [0]:
#Permitir a Apache Arrow convertir Pandas a pySpark DF:
spark.conf.set("spark.sql.execution.arrow.enabled", "true")

In [0]:
container = "yellowtaxis"
blobstorage = f"wasbs://{container}@{storage_account_name}.blob.core.windows.net"

for variable in d_amarillo.values():
    exec(variable)

    for clave, valor in d_amarillo.items():
        if variable == valor:

            #Código para exportar al almacenamiento previo (antes de transformar)
            dataframe = spark.createDataFrame(vars()[clave])
            
            dataframe.write.format("parquet").save(f"wasbs://yellowtaxis@pfcrudo.blob.core.windows.net/{clave}.parquet")

            print(clave, "exportado a .parquet")

---------------------------------------------------------------------------
Py4JJavaError                             Traceback (most recent call last)
<command-3007146002323688> in <cell line: 4>()
     11             dataframe = spark.createDataFrame(vars()[clave])
     12 
---> 13             dataframe.write.format("parquet").save(f"wasbs://yellowtaxis@pfcrudo.blob.core.windows.net/{clave}.parquet")
     14 
     15             print(clave, "exportado a .parquet")

/databricks/spark/python/pyspark/instrumentation_utils.py in wrapper(*args, **kwargs)
     46             start = time.perf_counter()
     47             try:
---> 48                 res = func(*args, **kwargs)
     49                 logger.log_success(
     50                     module_name, class_name, function_name, time.perf_counter() - start, signature

/databricks/spark/python/pyspark/sql/readwriter.py in save(self, path, format, mode, partitionBy, **options)
    966             self._jwrite.save()
    967        

In [0]:
#Exportar archivos pickles de los diccionarios de referencia al directorio del almacenamiento conveniente.

directorio_diccionarios = "/dbfs/d_amarillo.pickle"

amarillo = open(directorio_diccionarios, 'wb')
pickle.dump(d_amarillo, amarillo)
amarillo.close()